In [1]:
import pandas as pd
import altair as alt
import datetime

from IPython.display import Markdown

_ = alt.renderers.set_embed_options(theme="ggplot2", actions=False)


In [2]:
%%capture
%load_ext sql


::: {.callout-note}
This dashboard is using data from  [Glif nodes](https://lotus.filecoin.io/lotus/developers/glif-nodes/) `StateMarketDeals` periodic dump on S3 ([direct link](https://marketdeals.s3.amazonaws.com/StateMarketDeals.json.zst)).
:::

In [3]:
%%sql daily_data_on_chain <<
select
    date_trunc('day', sector_start_at) as date,
    sum(piece_size / 1024 ** 5) as onboarded_tibs
from filecoin_state_market_deals
where 
    sector_start_epoch > 0 and slash_epoch = -1
    and sector_start_at < current_date
    and sector_start_at > '2022-01-01'
group by 1
order by 1


In [4]:
df = daily_data_on_chain.DataFrame()
df["date"] = pd.to_datetime(df["date"])


::: {layout="[ [3, 2] ]"}

In [5]:
# | output: true

display(Markdown(f"### Data Onboarding Rate"))

c = (
    alt.Chart(df)
    .mark_bar()
    .encode(
        x=alt.X("date:T", title="Date"),
        y=alt.Y("onboarded_tibs:Q", title="Onboarded TiBs"),
        # color=alt.value("#55595c"),
        tooltip=[
            alt.Tooltip("date:T", title="Date"),
            alt.Tooltip("onboarded_tibs:Q", title="Onboarded TiBs"),
        ],
    )
    .properties(width="container", height=400)
)

# Make it zoomable and lock the y axis
# c = c.interactive(bind_y=False)

c.display()


### Data Onboarding Rate

alt.Chart(...)

In [6]:
# | output: true

display(Markdown(f"### Cumulative Data Onboarded"))

df["cumulative_data"] = df["onboarded_tibs"].cumsum()

alt.Chart(df).mark_line().encode(
    x=alt.X("date:T", title="Date"),
    y=alt.Y("cumulative_data:Q", title="Cumulative Data Onboarded (PiB)"),
    # color=alt.value("#55595c"),
    tooltip=[
        alt.Tooltip("date:T", title="Date"),
        alt.Tooltip("cumulative_data:Q", title="Cumulative Data Onboarded (PiB)"),
    ],
).properties(
    width="container",
    height=400,
)


### Cumulative Data Onboarded

alt.Chart(...)

:::

In [7]:
# | output: true

display(Markdown(f"### Onboarding Growth Rate (7 day rolling average)"))

df["growth_rate"] = df["onboarded_tibs"].pct_change(periods=7).rolling(7).mean() * 100
recent_df = df[df["date"] > "2022-01-01"]

alt.Chart(recent_df).mark_bar().encode(
    x=alt.X("date:T", title="Date"),
    y=alt.Y("growth_rate:Q", title="Onboarding Growth Rate"),
    # color=alt.value("#55595c"),
    tooltip=[
        alt.Tooltip("date:T", title="Date"),
        alt.Tooltip("growth_rate:Q", title="Onboarding Growth Rate"),
    ],
).properties(
    width="container",
    height=400,
)


### Onboarding Growth Rate (7 day rolling average)

alt.Chart(...)